In [23]:
#load data 

import pandas as pd
import pickle
from sklearn.cluster import KMeans
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

trip_data_file = "data/trip_data_4.csv"
trip_fare_file = "data/trip_fare_4.csv"

trips = pd.read_csv(trip_data_file, nrows=100000)
fares = pd.read_csv(trip_fare_file, nrows=100000)

full_tripdata = pd.merge(left=trips,right=fares, how='left', left_on=['medallion',' hack_license',' vendor_id',' pickup_datetime'], right_on=['medallion',' hack_license',' vendor_id',' pickup_datetime'])
full_tripdata = full_tripdata.drop_duplicates()

In [24]:
# Drop some un-wanted columns
full_tripdata = full_tripdata.drop(columns=['medallion',' hack_license',' vendor_id',' rate_code',' store_and_fwd_flag'])

In [25]:
#prepare time related column , weekday and hourofday
full_tripdata[" pickup_datetime"] = pd.to_datetime(full_tripdata[" pickup_datetime"])
full_tripdata[" dropoff_datetime"] = pd.to_datetime(full_tripdata[" dropoff_datetime"])

full_tripdata["weekday"] = full_tripdata[" pickup_datetime"].dt.dayofweek
full_tripdata["hourofday"] = full_tripdata[" pickup_datetime"].dt.hour


In [ ]:
full_tripdata.describe()

In [26]:
#remove outliers
measuredata = pd.concat([full_tripdata[' trip_time_in_secs'], 
                         full_tripdata[' pickup_longitude'],
                         full_tripdata[' pickup_latitude'],
                         full_tripdata[' dropoff_longitude'],
                         full_tripdata[' fare_amount'],
                         full_tripdata[' tip_amount'],
                        ],axis=1)

#remove data out of two standard deviation
full_tripdata = full_tripdata[(np.abs(stats.zscore(measuredata)) < 2).all(axis=1)]

#full_tripdata = full_tripdata.loc[((full_tripdata[' pickup_longitude'] < -72.0) & (full_tripdata[' pickup_latitude'] > 30.0)) 
                                  # | ((full_tripdata[' dropoff_latitude'] < -72.0) & (full_tripdata[' dropoff_latitude'] > 30.0))]

In [21]:
full_tripdata.describe()
full_tripdata.columns

Index([u' pickup_datetime', u' dropoff_datetime', u' passenger_count',
       u' trip_time_in_secs', u' trip_distance', u' pickup_longitude',
       u' pickup_latitude', u' dropoff_longitude', u' dropoff_latitude',
       u' payment_type', u' fare_amount', u' surcharge', u' mta_tax',
       u' tip_amount', u' tolls_amount', u' total_amount', u'weekday',
       u'hourofday', u'pickupLoc', u'dropoffLoc'],
      dtype='object')

In [27]:
#prepare location, use pre-trained k-mean model
model = pickle.load(open('location_cluster_kmean', 'rb'))
pickupLocations = full_tripdata.iloc[:,5:7]
#print pickupLocations.describe()
full_tripdata["pickupLoc"] = model.predict(pickupLocations.as_matrix())

dropoffLocations = full_tripdata.iloc[:,7:9]
full_tripdata["dropoffLoc"] = model.predict(dropoffLocations.as_matrix())
#print dropoffLocations.describe()

In [28]:
#drop all un-wanted columns
training = full_tripdata.drop(columns=[' pickup_datetime',
                                       ' dropoff_datetime',
                                       ' trip_time_in_secs',
                                       ' trip_distance',
                                       ' dropoff_longitude',
                                       ' dropoff_latitude',
                                       ' surcharge',
                                       ' mta_tax',
                                       ' tolls_amount',
                                       ' total_amount',
                                       ' passenger_count',
                                       ' payment_type',
                                       ' pickup_longitude',
                                       ' pickup_latitude'])

training.describe()

,fare_amount,tip_amount,weekday,hourofday,pickupLoc,dropoffLoc
count,91196.000000,91196.000000,91196.000000,91196.000000,91196.000000,91196.000000
mean,10.362697,1.056253,4.625334,10.267084,23.604687,24.214055
std,5.385798,1.290242,0.494550,9.301890,14.062651,14.017428
min,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.500000,0.000000,4.000000,1.000000,11.000000,12.000000
50%,9.000000,0.900000,5.000000,7.000000,23.000000,24.000000
75%,13.000000,1.900000,5.000000,22.000000,37.000000,38.000000
max,31.500000,5.800000,6.000000,23.000000,49.000000,49.000000


In [29]:
#column data type change 
training['weekday'] = training['weekday'].astype('category')
training['hourofday'] = training['hourofday'].astype('category')
training['pickupLoc'] = training['pickupLoc'].astype('category')
training['dropoffLoc'] = training['dropoffLoc'].astype('category')

In [30]:
#training and result data split
X = training.iloc[:,2:]
Y_fare = training.iloc[:,0]
Y_tip = training.iloc[:,1]

In [31]:
#convert category data to one-hot embedding
X = pd.get_dummies(X)

In [32]:
#dump the prepaired data

pickle.dump(X,open("X","wb"))
pickle.dump(Y_fare,open("Y_fare","wb"))
pickle.dump(X,open("Y_tip","wb"))